# Set up


In [35]:
!pip install mysqlclient

In [36]:
import MySQLdb

In [37]:
import getpass

In [38]:
import pandas as pd

In [39]:
db = MySQLdb.connect(host='sansa.cs.uoregon.edu', port=3331, user='ideas_user', passwd=getpass.getpass('db pass:\t'), db='ideas_db', charset='utf8', read_timeout=86400)

db pass:	··········


In [40]:
cursor = db.cursor()

In [41]:
def get_commits(proj_name):
    query = f'''
    select proj.id as project_id, proj.name as project_name,
	   proj.last_updated as project_last_updated, proj.source_url as project_source_url,
       proj.fork_of_id as project_fork_of_id, proj.child_of_id as project_child_of_id,
       c.id as commit_id, c.hash as commit_sha, c.datetime as commit_datetime, c.message as commit_message,
       c.branch as commit_branch, a.id as author_id, a.username as author_username, a.name as author_name,
       a.email as author_email, a.url as author_url, d.file_path as diff_file_path, d.language as diff_language,
       d.body as diff_body
    from project proj join commit c on(proj.id=c.project_id) join author a on(c.author_id=a.id)
        join diff d on (d.commit_id=c.id)
    where proj.name = '{proj_name}'
    '''
    cursor.execute(query)
    data = cursor.fetchall()
    cols = [col[0] for col in cursor.description]
    return pd.DataFrame(data, columns=cols)

In [42]:
def get_prs(proj_name):
    query = f'''
    select proj.id as project_id, proj.name as project_name,
	   proj.last_updated as project_last_updated, proj.source_url as project_source_url,
       proj.fork_of_id as project_fork_of_id, proj.child_of_id as project_child_of_id,
       p.id as pr_id, p.title as pr_title, p.description as pr_description, p.updated_at as pr_updated_at, p.created_at pr_created_at,
       p.locked as pr_locked, p.url as pr_url, p.number as pr_number, p.state as pr_state, p.merged_at as pr_merged_at,
       p.head_sha as pr_head_sha, a1.id as pr_author_id, a1.username as pr_author_username, a1.email as pr_author_email,
       a1.name as pr_author_name, a1.url as pr_author_url, m.id as milestone_id, m.title as milestone_title,
       m.description as milestone_description, m.state as milestone_state, m.due_on as milestone_due_on,
       m.created_at as milestone_created_at, m.updated_at as milestone_updated_at, l.id as pr_label_id, l.name as pr_label,
       a2.id as pr_assignee_id, a2.name as pr_assignee_name, a2.url as pr_assignee_url, a2.username as pr_assignee_username, a2.email as pr_assignee_email,
       ct.id as pr_sha_id, ct.sha as pr_sha, comm.id as pr_comment_id, comm.created_at as pr_comment_created_at, comm.updated_at as pr_comment_updated_at, comm.body as pr_comment_body,
	   a3.id as pr_comment_author_id, a3.name as pr_comment_author_name, a3.url as pr_comment_author_url, a3.username as pr_comment_author_username, a3.email as pr_comment_author_email,
       it.url as linked_issue_url
    from project proj join pr p on(p.project_id = proj.id) join author a1 on(a1.id=p.author_id) 
	   left join milestone m on(m.pr_id=p.id) left join pr_has_label phl on(phl.pr_id=p.id) left join label l on(l.id=phl.label_id)
       left join pr_has_assignee pha on(pha.pr_id=p.id) left join author a2 on(a2.id=pha.assignee_id)
       left join pr_has_commit phc on(phc.pr_id=p.id) left join commit_tag ct on(ct.id=phc.commit_id)
       left join comment comm on(comm.pr_id=p.id) left join author a3 on(a3.id=comm.author_id)
       left join pr_has_issue phi on(phi.pr_id=p.id) left join issue_tag it on(it.id=phi.issue_id)
    where proj.name = '{proj_name}';
    '''
    cursor.execute(query)
    data = cursor.fetchall()
    cols = [col[0] for col in cursor.description]
    return pd.DataFrame(data, columns=cols)
    

In [43]:
def get_issues(proj_name):
    query = f'''
    select proj.id as project_id, proj.name as project_name,
	   proj.last_updated as project_last_updated, proj.source_url as project_source_url,
       proj.fork_of_id as project_fork_of_id, proj.child_of_id as project_child_of_id,
	   i.id as issue_id, i.title as issue_title, i.description as issue_description, i.created_at as issue_created_at,
	   i.updated_at as issue_updated_at, i.closed_at as issue_closed_at, i.locked as issue_locked, i.state as issue_state,
       i.url as issue_url, i.number as issue_number, a1.username as issue_author_username, a1.email as issue_author_email,
       a1.name as issue_author_name, a1.url as issue_author_url, m.id as milestone_id, m.title as milestone_title,
       m.description as milestone_description, m.state as milestone_state, m.due_on as milestone_due_on,
       m.created_at as milestone_created_at, m.updated_at as milestone_updated_at, l.id as issue_label_id, l.name as issue_label,
       a2.id as issue_assignee_id, a2.name as issue_assignee_name, a2.url as issue_assignee_url, a2.username as issue_assignee_username, a2.email as issue_assignee_email,
       comm.id as issue_comment_id, comm.created_at as issue_comment_created_at, comm.updated_at as issue_comment_updated_at, comm.body as issue_comment_body,
	   a3.id as issue_comment_author_id, a3.name as issue_comment_author_name, a3.url as issue_comment_author_url, a3.username as issue_comment_author_username, a3.email as issue_comment_author_email
    from project proj join issue i on(proj.id=i.project_id) join author a1 on(a1.id=i.author_id) 
         left join milestone m on(m.issue_id=i.id) left join issue_has_label ihl on(ihl.issue_id=i.id) 
         left join label l on(l.id=ihl.label_id) left join issue_has_assignee iha on(iha.issue_id=i.id)
         left join author a2 on(iha.assignee_id=a2.id) left join comment comm on(comm.issue_id=i.id) left join author a3 on(a3.id=comm.author_id)
    where proj.name = '{proj_name}';'''
    cursor.execute(query)
    data = cursor.fetchall()
    cols = [col[0] for col in cursor.description]
    return pd.DataFrame(data, columns=cols)

In [44]:
def get_events(proj_name):
    query =f'''
    select p.id as project_id, p.source_url as project_url, p.name as project_name, p.last_updated as project_last_updated,
	   e.id as event_id, e.api_id as event_api_id, e.type as event_type, e.public as event_public, e.created_at as event_created_at,
       ea.id as event_actor_id, ea.actor_id as event_actor_api_id, ea.login as event_actor_login, ea.url as event_actor_url, 
       ea.avatar_url as event_actor_avatar_url, ea.gravatar_id as event_actor_gravatar_id,
       eo.id as event_org_id, eo.org_id as event_org_api_id, eo.login as event_org_login, eo.url as event_org_url, 
       eo.avatar_url as event_org_avatar_url, eo.gravatar_id as event_org_gravatar_id,
       er.id as event_repo_id, er.repo_id as event_repo_api_id, er.name as event_repo_name, er.url as event_repo_url,
       ep.id as event_payload_id, ep.action as event_payload_action, ep.ref as event_payload_ref, ep.ref_type as event_payload_ref_type,
       ep.master_branch as event_payload_master_branch,	ep.description as event_payload_description, ep.forkee_url as event_payload_forkee_url,
       ep.issue_url as event_payload_issue_url, ep.comment_url as event_payload_comment_url, ep.member_login as event_payload_member_login, 
       ep.pr_number as event_payload_pr_number,	ep.pr_url as event_payload_pr_url, ep.pr_review_url as event_payload_pr_review_url,	
       ep.push_id as event_payload_push_id,	ep.size as event_payload_size, ep.distinct_size as event_payload_distinct_size,	
       ep.head_sha as event_payload_head_sha, ep.before_sha as event_payload_before_sha, ep.release_url as event_payload_release_url,
       ep.effective_date as event_payload_effective_date, epage.id as event_page_id, epage.name as event_page_name, epage.title as event_page_title,
       epage.action as event_page_action, epage.sha as event_page_sha, epage.url event_page_url
    from project p join event e on(p.id=e.project_id) join event_actor ea on(e.actor_id=ea.id) join event_org eo on(e.org_id=eo.id)
			   join event_repo er on(e.repo_id=er.id) join event_payload ep on(e.payload_id=ep.id) left join event_has_page ehp on(ehp.payload_id=ep.id)
               left join event_page epage on (ehp.page_id=epage.id)
    where p.name = '{proj_name}';'''
    cursor.execute(query)
    data = cursor.fetchall()
    cols = [col[0] for col in cursor.description]
    return pd.DataFrame(data, columns=cols)

In [45]:
%%time
commit_df = get_commits('anl_test_repo')

CPU times: user 111 ms, sys: 77.7 ms, total: 188 ms
Wall time: 259 ms


In [46]:
%%time
pr_df = get_prs('anl_test_repo')

CPU times: user 130 ms, sys: 35.5 ms, total: 166 ms
Wall time: 241 ms


In [47]:
%%time
issue_df = get_issues('anl_test_repo')

CPU times: user 9.62 ms, sys: 0 ns, total: 9.62 ms
Wall time: 81.8 ms


In [48]:
%%time
event_df = get_events('anl_test_repo')

CPU times: user 5.41 ms, sys: 423 µs, total: 5.84 ms
Wall time: 82 ms


# Viewing the Data

## Commits

In [49]:
commit_df.shape

(12, 19)

In [50]:
commit_df.columns

Index(['project_id', 'project_name', 'project_last_updated',
       'project_source_url', 'project_fork_of_id', 'project_child_of_id',
       'commit_id', 'commit_sha', 'commit_datetime', 'commit_message',
       'commit_branch', 'author_id', 'author_username', 'author_name',
       'author_email', 'author_url', 'diff_file_path', 'diff_language',
       'diff_body'],
      dtype='object')

In [51]:
commit_df.head()

,project_id,project_name,project_last_updated,project_source_url,project_fork_of_id,project_child_of_id,commit_id,commit_sha,commit_datetime,commit_message,commit_branch,author_id,author_username,author_name,author_email,author_url,diff_file_path,diff_language,diff_body
0,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,230197,084c98c23b10309c29a8f9a12d681cfbd29bd986,2021-07-28 10:46:41,Create codeql-analysis.yml,* main\n remotes/origin/HEAD -> origin/main\n...,2247,fickas,None,fickas@cs.uoregon.edu,None,.github/workflows/codeql-analysis.yml,PLACEHOLDER,"+# For most projects, this workflow file will ..."
1,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,230197,084c98c23b10309c29a8f9a12d681cfbd29bd986,2021-07-28 10:46:41,Create codeql-analysis.yml,* main\n remotes/origin/HEAD -> origin/main\n...,2247,fickas,None,fickas@cs.uoregon.edu,None,.github/workflows/codeql-analysis.yml,PLACEHOLDER,"+# For most projects, this workflow file will ..."
2,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,230198,09d8d646e114b4a68bd2f216b8f036b1c60356f4,2021-07-26 10:11:04,Update test_file.py\n \n Added comment.,* main\n remotes/origin/HEAD -> origin/main\n...,2247,fickas,None,fickas@cs.uoregon.edu,None,test_file.py,PLACEHOLDER,+#print hello name!
3,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,230198,09d8d646e114b4a68bd2f216b8f036b1c60356f4,2021-07-26 10:11:04,Update test_file.py\n \n Added comment.,* main\n remotes/origin/HEAD -> origin/main\n...,2247,fickas,None,fickas@cs.uoregon.edu,None,test_file.py,PLACEHOLDER,+#print hello name!
4,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,230199,302dd37076dc9be51de80d2aaa3503298415d6ad,2021-07-26 10:10:31,Update test_file.py\n \n Added bang,* main\n remotes/origin/HEAD -> origin/main\n...,2247,fickas,None,fickas@cs.uoregon.edu,None,test_file.py,PLACEHOLDER,- return f'Hello {name}\n+ return f'Hello {n...


## Pull Requests

In [52]:
pr_df.shape

(15, 48)

In [53]:
pr_df.columns

Index(['project_id', 'project_name', 'project_last_updated',
       'project_source_url', 'project_fork_of_id', 'project_child_of_id',
       'pr_id', 'pr_title', 'pr_description', 'pr_updated_at', 'pr_created_at',
       'pr_locked', 'pr_url', 'pr_number', 'pr_state', 'pr_merged_at',
       'pr_head_sha', 'pr_author_id', 'pr_author_username', 'pr_author_email',
       'pr_author_name', 'pr_author_url', 'milestone_id', 'milestone_title',
       'milestone_description', 'milestone_state', 'milestone_due_on',
       'milestone_created_at', 'milestone_updated_at', 'pr_label_id',
       'pr_label', 'pr_assignee_id', 'pr_assignee_name', 'pr_assignee_url',
       'pr_assignee_username', 'pr_assignee_email', 'pr_sha_id', 'pr_sha',
       'pr_comment_id', 'pr_comment_created_at', 'pr_comment_updated_at',
       'pr_comment_body', 'pr_comment_author_id', 'pr_comment_author_name',
       'pr_comment_author_url', 'pr_comment_author_username',
       'pr_comment_author_email', 'linked_issue_url'],

In [54]:
pr_df.head()

,project_id,project_name,project_last_updated,project_source_url,project_fork_of_id,project_child_of_id,pr_id,pr_title,pr_description,pr_updated_at,pr_created_at,pr_locked,pr_url,pr_number,pr_state,pr_merged_at,pr_head_sha,pr_author_id,pr_author_username,pr_author_email,pr_author_name,pr_author_url,milestone_id,milestone_title,milestone_description,milestone_state,milestone_due_on,milestone_created_at,milestone_updated_at,pr_label_id,pr_label,pr_assignee_id,pr_assignee_name,pr_assignee_url,pr_assignee_username,pr_assignee_email,pr_sha_id,pr_sha,pr_comment_id,pr_comment_created_at,pr_comment_updated_at,pr_comment_body,pr_comment_author_id,pr_comment_author_name,pr_comment_author_url,pr_comment_author_username,pr_comment_author_email,linked_issue_url
0,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,7294,Bang bug - finally into main,,2021-07-29 21:14:13,2021-07-26 17:17:16,0,https://github.com/fickas/anl_test_repo/pull/4,4,MERGED,2021-07-26 17:18:06,09d8d646e114b4a68bd2f216b8f036b1c60356f4,3729,fickas,None,None,https://github.com/fickas,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,32504,302dd37076dc9be51de80d2aaa3503298415d6ad,NaN,NaT,NaT,None,NaN,None,None,None,None,https://github.com/fickas/anl_test_repo/issues/1
1,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,7294,Bang bug - finally into main,,2021-07-29 21:14:13,2021-07-26 17:17:16,0,https://github.com/fickas/anl_test_repo/pull/4,4,MERGED,2021-07-26 17:18:06,09d8d646e114b4a68bd2f216b8f036b1c60356f4,3729,fickas,None,None,https://github.com/fickas,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,32505,09d8d646e114b4a68bd2f216b8f036b1c60356f4,NaN,NaT,NaT,None,NaN,None,None,None,None,https://github.com/fickas/anl_test_repo/issues/1
2,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,7295,Goodbye enhancement to development,,2021-07-29 18:22:14,2021-07-29 18:21:48,0,https://github.com/fickas/anl_test_repo/pull/7,7,MERGED,2021-07-29 18:22:14,b6a43f27d7594c5a672f6e48cf0760773d09cb87,3729,fickas,None,None,https://github.com/fickas,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,32506,67dd6268097a7a945d15b7574b794e6c8dad5e6e,NaN,NaT,NaT,None,NaN,None,None,None,None,https://github.com/fickas/anl_test_repo/issues/6
3,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,7295,Goodbye enhancement to development,,2021-07-29 18:22:14,2021-07-29 18:21:48,0,https://github.com/fickas/anl_test_repo/pull/7,7,MERGED,2021-07-29 18:22:14,b6a43f27d7594c5a672f6e48cf0760773d09cb87,3729,fickas,None,None,https://github.com/fickas,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,32507,084c98c23b10309c29a8f9a12d681cfbd29bd986,NaN,NaT,NaT,None,NaN,None,None,None,None,https://github.com/fickas/anl_test_repo/issues/6
4,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,7295,Goodbye enhancement to development,,2021-07-29 18:22:14,2021-07-29 18:21:48,0,https://github.com/fickas/anl_test_repo/pull/7,7,MERGED,2021-07-29 18:22:14,b6a43f27d7594c5a672f6e48cf0760773d09cb87,3729,fickas,None,None,https://github.com/fickas,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,32508,b6a43f27d7594c5a672f6e48cf0760773d09cb87,NaN,NaT,NaT,None,NaN,None,None,None,None,https://github.com/fickas/anl_test_repo/issues/6


## Issues

In [55]:
issue_df.shape

(4, 43)

In [56]:
issue_df.columns

Index(['project_id', 'project_name', 'project_last_updated',
       'project_source_url', 'project_fork_of_id', 'project_child_of_id',
       'issue_id', 'issue_title', 'issue_description', 'issue_created_at',
       'issue_updated_at', 'issue_closed_at', 'issue_locked', 'issue_state',
       'issue_url', 'issue_number', 'issue_author_username',
       'issue_author_email', 'issue_author_name', 'issue_author_url',
       'milestone_id', 'milestone_title', 'milestone_description',
       'milestone_state', 'milestone_due_on', 'milestone_created_at',
       'milestone_updated_at', 'issue_label_id', 'issue_label',
       'issue_assignee_id', 'issue_assignee_name', 'issue_assignee_url',
       'issue_assignee_username', 'issue_assignee_email', 'issue_comment_id',
       'issue_comment_created_at', 'issue_comment_updated_at',
       'issue_comment_body', 'issue_comment_author_id',
       'issue_comment_author_name', 'issue_comment_author_url',
       'issue_comment_author_username', 'issue_

In [57]:
issue_df.head()

,project_id,project_name,project_last_updated,project_source_url,project_fork_of_id,project_child_of_id,issue_id,issue_title,issue_description,issue_created_at,issue_updated_at,issue_closed_at,issue_locked,issue_state,issue_url,issue_number,issue_author_username,issue_author_email,issue_author_name,issue_author_url,milestone_id,milestone_title,milestone_description,milestone_state,milestone_due_on,milestone_created_at,milestone_updated_at,issue_label_id,issue_label,issue_assignee_id,issue_assignee_name,issue_assignee_url,issue_assignee_username,issue_assignee_email,issue_comment_id,issue_comment_created_at,issue_comment_updated_at,issue_comment_body,issue_comment_author_id,issue_comment_author_name,issue_comment_author_url,issue_comment_author_username,issue_comment_author_email
0,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,3509,Missing bang,Add exclamation point to hello function.,2021-07-26 17:08:57,2021-07-26 17:18:07,2021-07-26 17:18:07,0,CLOSED,https://github.com/fickas/anl_test_repo/issues/1,1,fickas,None,None,https://github.com/fickas,None,None,None,None,None,None,None,45,bug,3729.0,None,https://github.com/fickas,fickas,None,None,None,None,None,None,None,None,None,None
1,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,3510,Add a second bang,hello should have two !!,2021-07-29 18:16:13,2021-07-29 18:16:13,2021-08-24 18:23:05,0,OPEN,https://github.com/fickas/anl_test_repo/issues/5,5,fickas,None,None,https://github.com/fickas,None,None,None,None,None,None,None,45,bug,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None
2,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,3511,New function goodbye,Add a function goodbye.,2021-07-29 18:18:58,2021-07-29 18:18:58,2021-08-24 18:23:05,0,OPEN,https://github.com/fickas/anl_test_repo/issues/6,6,fickas,None,None,https://github.com/fickas,None,None,None,None,None,None,None,46,enhancement,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None
3,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,3512,add going_to_lunch,add new function,2021-07-29 18:24:29,2021-07-29 18:24:29,2021-08-24 18:23:05,0,OPEN,https://github.com/fickas/anl_test_repo/issues/9,9,fickas,None,None,https://github.com/fickas,None,None,None,None,None,None,None,46,enhancement,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None


## Events

In [58]:
event_df.shape

(0, 51)

In [59]:
event_df.columns

Index(['project_id', 'project_url', 'project_name', 'project_last_updated',
       'event_id', 'event_api_id', 'event_type', 'event_public',
       'event_created_at', 'event_actor_id', 'event_actor_api_id',
       'event_actor_login', 'event_actor_url', 'event_actor_avatar_url',
       'event_actor_gravatar_id', 'event_org_id', 'event_org_api_id',
       'event_org_login', 'event_org_url', 'event_org_avatar_url',
       'event_org_gravatar_id', 'event_repo_id', 'event_repo_api_id',
       'event_repo_name', 'event_repo_url', 'event_payload_id',
       'event_payload_action', 'event_payload_ref', 'event_payload_ref_type',
       'event_payload_master_branch', 'event_payload_description',
       'event_payload_forkee_url', 'event_payload_issue_url',
       'event_payload_comment_url', 'event_payload_member_login',
       'event_payload_pr_number', 'event_payload_pr_url',
       'event_payload_pr_review_url', 'event_payload_push_id',
       'event_payload_size', 'event_payload_distinct_s

In [60]:
event_df.head()

,project_id,project_url,project_name,project_last_updated,event_id,event_api_id,event_type,event_public,event_created_at,event_actor_id,event_actor_api_id,event_actor_login,event_actor_url,event_actor_avatar_url,event_actor_gravatar_id,event_org_id,event_org_api_id,event_org_login,event_org_url,event_org_avatar_url,event_org_gravatar_id,event_repo_id,event_repo_api_id,event_repo_name,event_repo_url,event_payload_id,event_payload_action,event_payload_ref,event_payload_ref_type,event_payload_master_branch,event_payload_description,event_payload_forkee_url,event_payload_issue_url,event_payload_comment_url,event_payload_member_login,event_payload_pr_number,event_payload_pr_url,event_payload_pr_review_url,event_payload_push_id,event_payload_size,event_payload_distinct_size,event_payload_head_sha,event_payload_before_sha,event_payload_release_url,event_payload_effective_date,event_page_id,event_page_name,event_page_title,event_page_action,event_page_sha,event_page_url


# Functions


## View Events History


In [61]:
def view_event_history(event_df, first=100):
    payload_cols = [col for col in event_df if col.startswith('event_payload')]
    for i, (_, row) in enumerate(event_df.sort_values('event_created_at', ascending=False).iterrows()):
        if i == first:
            break 
        print(row['event_id'], row['event_type'], row['event_created_at'])
        for col in payload_cols:
            if row[col] and not pd.isnull(row[col]):
                print(f'\t{col}: {row[col]}')
        print('-'*80)

In [86]:
view_event_history(event_df)

## List Open Issues

In [63]:
def list_open_issues(issue_df):
    issue_df = issue_df.drop_duplicates(subset=['issue_label_id'])
    
    issue_df = issue_df.query('issue_state == "OPEN"')
    issue_df = issue_df[['issue_id', 'issue_title', 'issue_description', 'issue_label',
             'issue_author_username', 'issue_author_email', 'issue_author_name', 'issue_author_url',
             'issue_assignee_username', 'issue_assignee_email', 'issue_assignee_name', 'issue_assignee_url']]
    issue_df['infer_bug'] = 'bug' in issue_df['issue_title'].str.lower() or 'bug' in issue_df['issue_description'].str.lower()
    issue_df = issue_df.reset_index(drop=True)
    return issue_df

In [64]:
list_open_issues(issue_df)

,issue_id,issue_title,issue_description,issue_label,issue_author_username,issue_author_email,issue_author_name,issue_author_url,issue_assignee_username,issue_assignee_email,issue_assignee_name,issue_assignee_url,infer_bug
0,3511,New function goodbye,Add a function goodbye.,enhancement,fickas,None,None,https://github.com/fickas,None,None,None,None,False


## Developer Info

In [65]:
def get_developer_info(username, pr_df, issue_df, commit_df):
    pr_df = pr_df.query(f'pr_author_username == "{username}" and pr_state == "OPEN"')
    pr_df = pr_df.drop_duplicates(subset=['pr_id'])
    pr_df = pr_df[['pr_id', 'pr_title', 'pr_description', 'pr_sha']]
    issue_df = issue_df.query(f'issue_author_username == "{username}" and issue_state == "OPEN"')
    issue_df = issue_df.drop_duplicates(subset=['issue_id'])
    issue_df = issue_df[['issue_id', 'issue_title', 'issue_description']]
    pr_df = pr_df.reset_index(drop=True)
    issue_df = issue_df.reset_index(drop=True)
    return pr_df, issue_df

In [84]:
dev_pr_df, dev_issue_df = get_developer_info('fickas', pr_df, issue_df, commit_df)

In [85]:
dev_pr_df

,pr_id,pr_title,pr_description,pr_sha
0,7297,Update test_file.py,Added going to lunch.,f89edf704d5bb032a3e5195b2bb199bc511c941f


In [68]:
# missing commit info?
for _, row in dev_pr_df.iterrows():
        a = commit_df.query(f'commit_sha == "{row["pr_sha"]}"')
        display(a)

In [69]:
dev_issue_df

,issue_id,issue_title,issue_description


## Pairing Commits to Issues

In [70]:
import re

In [71]:
pattern1 = re.compile(r'Issue #(\d+)')

In [72]:
pattern2 = re.compile(r'^\(Issue #(\d+)\)')

In [73]:
df = commit_df[['commit_sha', 'commit_message']].drop_duplicates(subset=['commit_sha']).reset_index(drop=True)

In [74]:
data = [[], [], []]
for message in df['commit_message']:
    match = re.findall(pattern1, message)
    data[0].append(len(match) != 0)
    data[1].append(list(set(match)))
    strict_match = re.findall(pattern2, message)
    data[2].append(len(strict_match) == 1)
df['issue_related'] = data[0]
df['strict_issue_related'] = data[2]
df['issue_numbers'] = data[1]

In [75]:
df.query('issue_related == True').reset_index(drop=True)

,commit_sha,commit_message,issue_related,strict_issue_related,issue_numbers


In [76]:
df.query('strict_issue_related == True').reset_index(drop=True)

,commit_sha,commit_message,issue_related,strict_issue_related,issue_numbers


## Grabing Linked Issues and Commits for a Pull Request

In [77]:
def find_linked_pr_info(pr_url):
    df = pr_df.query(f'pr_url == "{pr_url}"')
    linked_shas = df.pr_sha.unique()
    linked_commits_df = commit_df.query('commit_sha in @linked_shas')
    linked_issues = df.linked_issue_url.unique()
    linked_issues_df = issue_df.query('issue_url in @linked_issues')
    return (linked_commits_df, linked_issues_df)

In [81]:
linked_commits_df, linked_issues_df = find_linked_pr_info('https://github.com/fickas/anl_test_repo/pull/4')

In [82]:
linked_commits_df

,project_id,project_name,project_last_updated,project_source_url,project_fork_of_id,project_child_of_id,commit_id,commit_sha,commit_datetime,commit_message,commit_branch,author_id,author_username,author_name,author_email,author_url,diff_file_path,diff_language,diff_body
2,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,230198,09d8d646e114b4a68bd2f216b8f036b1c60356f4,2021-07-26 10:11:04,Update test_file.py\n \n Added comment.,* main\n remotes/origin/HEAD -> origin/main\n...,2247,fickas,None,fickas@cs.uoregon.edu,None,test_file.py,PLACEHOLDER,+#print hello name!
3,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,230198,09d8d646e114b4a68bd2f216b8f036b1c60356f4,2021-07-26 10:11:04,Update test_file.py\n \n Added comment.,* main\n remotes/origin/HEAD -> origin/main\n...,2247,fickas,None,fickas@cs.uoregon.edu,None,test_file.py,PLACEHOLDER,+#print hello name!
4,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,230199,302dd37076dc9be51de80d2aaa3503298415d6ad,2021-07-26 10:10:31,Update test_file.py\n \n Added bang,* main\n remotes/origin/HEAD -> origin/main\n...,2247,fickas,None,fickas@cs.uoregon.edu,None,test_file.py,PLACEHOLDER,- return f'Hello {name}\n+ return f'Hello {n...
5,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,230199,302dd37076dc9be51de80d2aaa3503298415d6ad,2021-07-26 10:10:31,Update test_file.py\n \n Added bang,* main\n remotes/origin/HEAD -> origin/main\n...,2247,fickas,None,fickas@cs.uoregon.edu,None,test_file.py,PLACEHOLDER,- return f'Hello {name}\n+ return f'Hello {n...


In [83]:
linked_issues_df

,project_id,project_name,project_last_updated,project_source_url,project_fork_of_id,project_child_of_id,issue_id,issue_title,issue_description,issue_created_at,issue_updated_at,issue_closed_at,issue_locked,issue_state,issue_url,issue_number,issue_author_username,issue_author_email,issue_author_name,issue_author_url,milestone_id,milestone_title,milestone_description,milestone_state,milestone_due_on,milestone_created_at,milestone_updated_at,issue_label_id,issue_label,issue_assignee_id,issue_assignee_name,issue_assignee_url,issue_assignee_username,issue_assignee_email,issue_comment_id,issue_comment_created_at,issue_comment_updated_at,issue_comment_body,issue_comment_author_id,issue_comment_author_name,issue_comment_author_url,issue_comment_author_username,issue_comment_author_email
0,30,anl_test_repo,2021-07-30 21:43:15,https://github.com/fickas/anl_test_repo.git,None,None,3509,Missing bang,Add exclamation point to hello function.,2021-07-26 17:08:57,2021-07-26 17:18:07,2021-07-26 17:18:07,0,CLOSED,https://github.com/fickas/anl_test_repo/issues/1,1,fickas,None,None,https://github.com/fickas,None,None,None,None,None,None,None,45,bug,3729.0,None,https://github.com/fickas,fickas,None,None,None,None,None,None,None,None,None,None
